# Sentiment Analysis

## Import statements

In [1]:
import pandas as pd
import re
from functools import reduce
from textblob import TextBlob

/Users/laureneterno/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Save excel files from data cleaning stage into DataFrames and name the timestamp column

In [2]:
nike = pd.read_excel('nike.xlsx')
starbucks = pd.read_excel('starbucks.xlsx')
target = pd.read_excel('target.xlsx')

nike = nike.rename(columns = {'Unnamed: 0': 'timestamp'})
starbucks = starbucks.rename(columns = {'Unnamed: 0': 'timestamp'})
target = target.rename(columns = {'Unnamed: 0': 'timestamp'})

## Define getPolarity function that uses the TextBlob library to perform sentiment analysis on the comments

In [3]:
def getPolarity(text):
   return TextBlob(text).sentiment.polarity

## Define sentiment analysis function that cleans the string data and assigns each date a polarity score using the TextBlob library

In [4]:
def sentiment_analysis(df):
    list = []
    for x in df['comments']:
    
        # changes all of the characters to lowercase
        x = str.lower(x)
    
        # removes all special characters
        x = re.sub(r'[^a-zA-Z0-9\s]+', '', x)
    
        # adds the modified column values to a list
        list.append(x)

        # adds the nike_list to a new dataframe
        new_df = pd.DataFrame(list, columns =['comments'])

    # adds the timestamps to the new dataframe
    new_df = new_df.join(df['timestamp'])

    # performs sentiment analysis on the comments from each day
    new_df['polarity_score'] = new_df['comments'].apply(getPolarity)

    # rearrange column order
    new_df = new_df[['timestamp', 'polarity_score','comments']]

    # set timestamp as index
    new_df.set_index('timestamp')
    
    return new_df

## Creates the new dataframe using the sentiment analysis function on each company's dataframe

In [5]:
nike = sentiment_analysis(nike)

In [6]:
starbucks = sentiment_analysis(starbucks)

In [7]:
target = sentiment_analysis(target)

# Machine Learning

## Import Statements

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error

2023-12-06 17:23:28.381294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
nike_stock = pd.read_csv('nike_stock.csv')
starbucks_stock = pd.read_csv('starbucks_stock.csv')
target_stock = pd.read_csv('target_stock.csv')

In [10]:
def combine_and_clean(stock, sa):
    
    # drop columns not needed from initial data retreival
    stock = stock.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis = 1)
    
    # change columns to datetime type
    sa['timestamp'] = pd.to_datetime(sa['timestamp'])
    stock['Date'] = pd.to_datetime(stock['Date'])
    
    # perform an inner join to combine the dataframes by the dates available for both the sentiment analysis and the stock data
    sa = sa.merge(stock, how = 'inner', left_on='timestamp', right_on = 'Date')
    
    # drop the date column as it is the same as the timestamp column (since the join was performed) and drop the comments column as the sentiment analysis was already completed
    sa = sa.drop(['Date', 'comments'], axis = 1)
    
    # rename the Close column to close to match the lowercase in the other column names
    sa = sa.rename(columns = {'Close': 'close'})
    
    # rename the timstamp column to date
    sa = sa.rename(columns = {'timestamp':'date'})
    
    # Convert 'date' to numerical feature (number of days since the start)
    sa['days_since_start'] = (sa['date'] - sa['date'].min()).dt.days
        
    #return the cleaned dataframe
    return sa

In [11]:
nike = combine_and_clean(nike_stock, nike)
starbucks = combine_and_clean(starbucks_stock, starbucks)
target = combine_and_clean(target_stock, target)

In [12]:
def model(df):
    # feature selection
    features = df[['days_since_start', 'polarity_score', 'close']].values
    
    # normalize data
    # scales values between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(features)

    # Define nested function to create dataset with input features and target variable
    def create_dataset(data, time_steps=1):
        # x represents the input features
        # y represents the output variables
        dataX, dataY = [], []
        for i in range(len(data) - time_steps):
            a = data[i:(i + time_steps), :]
            dataX.append(a)
            dataY.append(data[i + time_steps, 2])  # 'close' is the third column
        return np.array(dataX), np.array(dataY)

    # determines the sequence length for each input
    time_steps = 10

    # create dataset
    x, y = create_dataset(scaled_data, time_steps)

    # split data into training and testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # Build LSTM model
    # 2 LSTM layers
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(LSTM(units=50))
    # Dense output layer
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # train the model
    model.fit(x_train, y_train, epochs=50, batch_size=32)

    # evaluate the model
    test_loss = model.evaluate(x_test, y_test)

    # define variable for test_loss output
    test_loss_output = f'Test Loss: {test_loss}'

    # make predictions
    predictions = model.predict(x_test)

    # inverse transform the predictions to get the original scale
    predictions = scaler.inverse_transform(np.concatenate((x_test[:, -1, 0:2], predictions.reshape(-1, 1)), axis=1))[:, 2]
    
    # evaluate the predictions
    actual_close = scaler.inverse_transform(np.concatenate((x_test[:, -1, 0:2], y_test.reshape(-1, 1)), axis=1))[:, 2]
  
    mae = mean_absolute_error(actual_close, predictions)
    mae_output = f'Mean Absolute Error: {mae}'
    return predictions, test_loss_output, mae_output

In [13]:
model(nike)

Epoch 1/50
3/3 [==============================] - 3s 11ms/step - loss: 0.1967
Epoch 2/50
3/3 [==============================] - 0s 14ms/step - loss: 0.0788
Epoch 3/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0379
Epoch 4/50
3/3 [==============================] - 0s 13ms/step - loss: 0.0540
Epoch 5/50
3/3 [==============================] - 0s 14ms/step - loss: 0.0484
Epoch 6/50
3/3 [==============================] - 0s 12ms/step - loss: 0.0355
Epoch 7/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0321
Epoch 8/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0341
Epoch 9/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0332
Epoch 10/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0295
Epoch 11/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0271
Epoch 12/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0273
Epoch 13/50
3/3 [==============================] - 0s 8ms/step - 

(array([107.9588487 , 100.76664805, 103.46373167, 102.26437955,
        100.35074226, 110.78858271,  99.94259358, 101.39324114,
         99.65561327, 111.14663286, 114.03089743, 110.13729977,
        107.01866399, 110.25785566, 112.81952843, 109.88412964,
        108.08888113, 109.01713828]),
 'Test Loss: 0.019133079797029495',
 'Mean Absolute Error: 3.714160731373678')

In [14]:
model(starbucks)

Epoch 1/50
3/3 [==============================] - 2s 9ms/step - loss: 0.2362
Epoch 2/50
3/3 [==============================] - 0s 9ms/step - loss: 0.1175
Epoch 3/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0633
Epoch 4/50
3/3 [==============================] - 0s 11ms/step - loss: 0.0786
Epoch 5/50
3/3 [==============================] - 0s 13ms/step - loss: 0.0662
Epoch 6/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0498
Epoch 7/50
3/3 [==============================] - 0s 10ms/step - loss: 0.0478
Epoch 8/50
3/3 [==============================] - 0s 11ms/step - loss: 0.0491
Epoch 9/50
3/3 [==============================] - 0s 9ms/step - loss: 0.0457
Epoch 10/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0403
Epoch 11/50
3/3 [==============================] - 0s 8ms/step - loss: 0.0412
Epoch 12/50
3/3 [==============================] - 0s 7ms/step - loss: 0.0435
Epoch 13/50
3/3 [==============================] - 0s 7ms/step - los

(array([ 99.97705615,  95.33075002,  93.91570325, 101.77887513,
         99.31157707,  99.59448968, 101.88321914,  97.65806166,
        103.36782639,  96.08067868, 106.12850961, 101.13691055,
         94.30035993, 100.1136909 , 102.85073618, 100.09636866,
         99.29925773, 100.3038583 , 101.95808246, 101.99488234,
         96.50981992, 101.16484889]),
 'Test Loss: 0.017128417268395424',
 'Mean Absolute Error: 2.5036735496756646')

In [15]:
model(target)

Epoch 1/50
4/4 [==============================] - 2s 7ms/step - loss: 0.0771
Epoch 2/50
4/4 [==============================] - 0s 9ms/step - loss: 0.0577
Epoch 3/50
4/4 [==============================] - 0s 9ms/step - loss: 0.0305
Epoch 4/50
4/4 [==============================] - 0s 9ms/step - loss: 0.0335
Epoch 5/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0281
Epoch 6/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0306
Epoch 7/50
4/4 [==============================] - 0s 6ms/step - loss: 0.0251
Epoch 8/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0271
Epoch 9/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0266
Epoch 10/50
4/4 [==============================] - 0s 6ms/step - loss: 0.0237
Epoch 11/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0258
Epoch 12/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0245
Epoch 13/50
4/4 [==============================] - 0s 7ms/step - loss: 0.

(array([136.12982427, 151.94890613, 149.82947315, 124.48332776,
        130.53060464, 138.29899723, 117.34501224, 135.72253328,
        158.07901283, 159.28151861, 128.29168013, 132.3280207 ,
        124.09910256, 128.10778522, 159.82960354, 131.95675565,
        162.32041325, 111.35540147, 148.79044479, 155.14418168,
        163.11406655, 122.51446574, 121.53697015, 123.68260372,
        116.17330131, 135.74623269, 128.00835679]),
 'Test Loss: 0.011379473842680454',
 'Mean Absolute Error: 5.6589159572946')